In [8]:
from general import *

In [9]:
class PostureNeuron():
    NeuronState = namedtuple('NeuronState', ['V', 'w', 'spk', 'I'])

    def __init__(self, parameters):
        super(PostureNeuron, self).__init__()
        self.tau = parameters['tau']
        self.V_R = parameters['V_R']
        self.V_T = parameters['V_T']
        self.w = parameters['w']
        self.n = parameters['n']
        self.N_input = parameters['N_input']
        self.dt = parameters['dt']
        self.state = None

    def initialize_state(self):
        self.state = None

    def forward(self, input):
        if self.state is None:
            self.state = self.NeuronState(V=np.full((self.n,), self.V_R),
                                          w=np.array(self.w),
                                          spk=np.zeros(self.n),
                                          I=np.zeros((self.n, self.N_input)))
            
        V = self.state.V
        w = self.state.w
        I = self.state.I
        
        V += self.dt * (self.V_R - V) / self.tau
        V += np.sum(w * input, axis=-1)

        spk = np.heaviside(V - self.V_T, 0)
        V = (1 - spk) * V + spk * self.V_R

        self.state = self.NeuronState(V=V, w=w, spk=spk, I=I)

        return V, spk

In [10]:
with open('data/spike_primitive', 'rb') as file:
    spike_primitive = pickle.load(file) 
    
with open('data/joint_angles_' + parameters['joint_string'], 'rb') as file:
    joint_angles = pickle.load(file)
    
posture_parameters = {'tau': 50e-3, 'V_T': -50e-3, 'V_R': -70e-3, 'n': 2, 'w': 0, 'N_input': 672, 'dt': constants['dt']}

posture_neuron = PostureNeuron(posture_parameters)
posture_neuron.initialize_state()

In [11]:
weights = np.zeros((672, 2))

for j in range(672):
    incidence_binned = np.zeros(2)

    for i in range(9):

        spike_train = spike_primitive[:, j, i]
        incidence = spike_train * pitch[:, i]
        incidence[incidence == 0] = np.nan
        incidence_binned += np.histogram(incidence, bins=2, range=(-180, 200))[0]

    ratio = incidence_binned[1] / incidence_binned[0]

    if ratio > ratios[l]:
        weights[j, 0] = w_1_list[m]

    if ratio < 1 / ratios[l]:
        weights[j, 1] = w_1_list[m]

NameError: name 'N_SIMULATIONS' is not defined